In [ ]:
# Case de Predição de Fraudes em Transações de Cartão de Crédito 💳
'''
## Contexto:

Você é um(a) cientista de dados em uma instituição financeira renomada. 
A empresa está enfrentando um aumento significativo nas transações fraudulentas de cartões de crédito, 
o que está prejudicando a confiança dos clientes e causando prejuízos financeiros. 
Sua tarefa é desenvolver um modelo preditivo que possa identificar transações fraudulentas com alta precisão, 
minimizando assim o impacto financeiro e protegendo a reputação da instituição.

## Base de Dados:

Você tem acesso à base de dados "**fraud_dataset.csv**", que contém informações detalhadas sobre as transações
de cartões de crédito. 

## Objetivo:

Desenvolver um modelo preditivo utilizando o algoritmo de Regressão Logística com regularização L2 para prever
se uma transação de cartão de crédito é fraudulenta ou não.

**Realize os seguintes passos na sua entrega:**

- Pré-processamento de Dados:
  - Verifique se a base de dados contém informações nulas.
  - Considere a padronização de variáveis numéricas.
- Divisão da Base de Dados:
  - Separe a base de dados utilizando a técnica hold out, mantendo uma proporção adequada para garantir 
  a representatividade dos dados.
  - Caso a base de dados tenha algum tipo de desequilíbrio, aplique uma técnica de oversampling para 
  equilibrar as classes.
- Modelagem:
  - Implemente o algoritmo de Regressão Logística com regularização L2.
  - Utilize a técnica de validação cruzada k-fold (com 5 folds) para avaliar o desempenho do modelo.
- Avaliação do Modelo:
  - Avalie o modelo utilizando métricas como precisão, recall, F1-score e área sob a curva ROC (AUC-ROC).
- Interpretação e Comunicação de Resultados:
  - Comunique os resultados de forma clara e acessível para as partes interessadas, destacando a eficácia 
  do modelo na identificação de transações fraudulentas.
  '''

In [65]:
import pandas as pd

In [67]:
df = pd.read_csv('fraud_dataset.csv')
df.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


In [69]:
#Verifique se a base de dados contém informações nulas.
df.isnull().sum()

distance_from_home                0
distance_from_last_transaction    0
ratio_to_median_purchase_price    0
repeat_retailer                   0
used_chip                         0
used_pin_number                   0
online_order                      0
fraud                             0
dtype: int64

In [71]:
df.shape

(1000000, 8)

In [73]:
#Verificação de desequilíbrio de classe - fraud
round(df['fraud'].value_counts(normalize=True)*100,2)
#há desequilíbrio

fraud
0.0    91.26
1.0     8.74
Name: proportion, dtype: float64

In [75]:
df.columns

Index(['distance_from_home', 'distance_from_last_transaction',
       'ratio_to_median_purchase_price', 'repeat_retailer', 'used_chip',
       'used_pin_number', 'online_order', 'fraud'],
      dtype='object')

In [77]:
#separar antes da normalização
#técnica de hold out( 80%treino 20%teste)
from sklearn.model_selection import train_test_split
X = df.drop('fraud', axis=1)
y = df['fraud']
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2,stratify=y, random_state=7)


In [79]:
print(X_train.shape, X_test.shape)

(800000, 7) (200000, 7)


In [81]:
#Considere a padronização de variáveis numéricas.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(X_train) #para não deixar meus dados enviezados

X_train_escalonado = scaler.transform(X_train)
X_test_escalonado = scaler.transform(X_test)


In [83]:
#Caso a base de dados tenha algum tipo de desequilíbrio, 
#aplique uma técnica de oversampling para equilibrar as classes.

from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority', random_state=7)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_escalonado,y_train)

In [84]:
print(pd.Series(y_train_resampled).value_counts(normalize=True)*100)

fraud
0.0    50.0
1.0    50.0
Name: proportion, dtype: float64


In [95]:
#Implemente o algoritmo de Regressão Logística com regularização L2.
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

from sklearn.model_selection import StratifiedKFold,cross_val_score
from sklearn.linear_model import LogisticRegression

modelo = LogisticRegression(penalty='l2')

#Utilize a técnica de validação cruzada k-fold (com 5 folds) para avaliar o desempenho do modelo.
cv = StratifiedKFold(n_splits=5,shuffle=True, random_state=7)
scores = cross_val_score(modelo, X_train_resampled, y_train_resampled, cv=cv, scoring='roc_auc')


In [96]:
print(scores.mean())

0.9793301515239321


In [101]:
modelo.fit(X_train_resampled, y_train_resampled)
y_pred = modelo.predict(X_test_escalonado)

In [109]:
#Avalie o modelo utilizando métricas como precisão, recall, F1-score e área sob a curva ROC (AUC-ROC)
from sklearn.metrics import classification_report, roc_auc_score

auc_score=roc_auc_score(y_test,y_pred)
print(classification_report(y_test,y_pred))
print(auc_score.mean())

              precision    recall  f1-score   support

         0.0       1.00      0.93      0.96    182519
         1.0       0.58      0.95      0.72     17481

    accuracy                           0.93    200000
   macro avg       0.79      0.94      0.84    200000
weighted avg       0.96      0.93      0.94    200000

0.9430003032089769


In [ ]:
#Comunique os resultados de forma clara e acessível para as partes interessadas, destacando a eficácia 
#do modelo na identificação de transações fraudulentas.

#O modelo apresentou que consegue distinguir o que é fraude ou não, mas ainda não está tão acertivo. 
#há outros modelos que seriam melhores para desequilíbrio de classes: Modelos Baseados em Árvore (como Random Forest, XGBoost)
#Capturam relações não lineares e interações entre variáveis.
#Lidam bem com variáveis categóricas, valores ausentes, outliers.
#Têm parâmetros dedicados ao balanceamento (como scale_pos_weight, class_weight).
#Tendem a ter melhor desempenho em métricas como F1-score, Recall e AUC em dados desbalanceados.